In [4]:
import os

In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Codewtithdips/end-to-end-ml-pipeline.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Codewtithdips"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1f2ce87caa99243e1f28b650b8e95967bc9a5048"

In [3]:
%pwd

'c:\\end to end\\end-to-end-ml-pipeline\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\end to end\\end-to-end-ml-pipeline'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [8]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Codewtithdips/end-to-end-ml-pipeline.mlflow"
        )


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import numpy as np
import joblib
from pathlib import Path
from src.datascience.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_tracking_uri(self.config.mlflow_uri)

        with mlflow.start_run():

            preds = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, preds)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # MANUAL MODEL SAVING (DAGSHUB COMPATIBLE)
            joblib.dump(model, "model.joblib")
            mlflow.log_artifact("model.joblib", artifact_path="model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e


[2025-12-08 12:46:59,377 : INFO] : common : yaml file: config\config.yaml loaded successfully]
[2025-12-08 12:46:59,384 : INFO] : common : yaml file: params.yaml loaded successfully]
[2025-12-08 12:46:59,390 : INFO] : common : yaml file: schema.yaml loaded successfully]
[2025-12-08 12:46:59,393 : INFO] : common : created directory at: artifacts]
[2025-12-08 12:46:59,395 : INFO] : common : created directory at: artifacts/model_evaluation]
[2025-12-08 12:47:02,247 : INFO] : common : json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run handsome-flea-399 at: https://dagshub.com/Codewtithdips/end-to-end-ml-pipeline.mlflow/#/experiments/0/runs/3a27bce4aa9f4fc083484175df7c5e93
🧪 View experiment at: https://dagshub.com/Codewtithdips/end-to-end-ml-pipeline.mlflow/#/experiments/0
